## ILUMINACION

## Usando imagenes de entrenamiento con iluminacion adecuada, calculamos un promedio de RGB, que usaremos para evaluar nuevas imagenes y tratar de acercarlas a este valor promedio de RGB 

In [1]:
import numpy as np
import statistics
from PIL import Image 
import zipfile
import gdown
import os
import tkinter as tk
from tkinter import filedialog
import cv2
from skimage.color import rgba2rgb
from skimage import data

In [2]:


url = 'https://drive.google.com/uc?id=1JWdBLyIptl26i1tkYowRlxR394WxHtho'
output = '/tmp/LEARNRGB.zip'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1JWdBLyIptl26i1tkYowRlxR394WxHtho
To: /tmp/LEARNRGB.zip
100%|██████████| 7.70M/7.70M [00:01<00:00, 5.83MB/s]


'/tmp/LEARNRGB.zip'

In [3]:
local_zip = '/tmp/LEARNRGB.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

IMG_dir = '/tmp/LEARNRGB'


In [4]:

def AVRGB(img):

    R_VAR = [] 
    G_VAR = []
    B_VAR = []

    for s in img:
        image = s
         #calculamos el valor promedio de RGB para tener una idea en donde 
         # deberian estar nuestras imagenes con mala iluminacion 
        AverageV = np.mean(image)
    return AverageV
im = []

import glob

#cargamos 6 imagenes de entrenamiento con buena iluminacion
image_list = []
for filename in glob.glob(IMG_dir + '/*.jpg'):
    im.append(Image.open(filename))
    

#nuestro promedio de RGB "Ideal" donde queremos que nuestras imagenes esten 
RGBAVG = AVRGB(im)

print("Promedio de RGB de nuestras imagenes de entrenamiento:" + str(RGBAVG))

print("Clase 1 valores de: 0 a " + str(RGBAVG-10))
print("Clase 2 valores de: " + str(RGBAVG+10) + " a 255")
print("Clase 3 valores de: " + str(RGBAVG-10) + " a " + str(RGBAVG + 10))


Promedio de RGB de nuestras imagenes de entrenamiento:70.60332139756945
Clase 1 valores de: 0 a 60.603321397569445
Clase 2 valores de: 80.60332139756945 a 255
Clase 3 valores de: 60.603321397569445 a 80.60332139756945


## Funcion que modificara nuestra imagen basado en valores de RGB vistos en otras imagenes 

In [5]:
#  Usando el valor promedio para RGB "RGBAVG" en el paso anterior podemos usar ese valor para tratar de aproximar 
#  nuestras imagenes con falta de iluminacion y se acerquen a ese valor,tambien estableceremos un 
#  rango +-10 de RGBAVG si una nueva imagen entra en este rango la imagen no se modificara ya que esta en 
#  un rango acceptable, de lo contrario si esta sobre el rango se disminuira la iluminacion 
#  para visualizarla mejor



def FIX_MY_IMAGE(imageinput):
    image = imageinput
    ih = image.height
    iw = image.width
   

    R_VAR = [] 
    G_VAR = []
    B_VAR = []

    for i in range(ih):
        for ii in range(iw):
            R_VAR.append(image.getpixel((ii,i))[0])
            G_VAR.append(image.getpixel((ii,i))[1])
            B_VAR.append(image.getpixel((ii,i))[2])

  

    R_VARC = statistics.mean(R_VAR)
    G_VARC = statistics.mean(G_VAR)
    B_VARC = statistics.mean(B_VAR)
   
  
    # calculamos el promedio RGB de la imagen que estamos analizando 
    ENTRYRGB = statistics.mean([statistics.mean(R_VAR),statistics.mean(G_VAR), statistics.mean(B_VAR)])
    
    print("RGB promedio antes de procesar:" + str(ENTRYRGB))
   

    # si nuestra imagen esta por debajo de RGBAVG - 10 entonces quiere decir que nuestra imagen esta muy obscura 
    # y calculamos el numero que multiplicado por nuestro promedio nos acerque al valor de RGBAVG 
    
    # Clase 1
    
    if ENTRYRGB <= (RGBAVG -10):
        R_VARC = (RGBAVG -10) / (statistics.mean(R_VAR)+1)
        G_VARC = (RGBAVG -10) / (statistics.mean(G_VAR)+1)
        B_VARC = (RGBAVG -10) / (statistics.mean(B_VAR)+1)


    # si nuestra imagen esta por arriba de RGBAVG + 10, entonces quiere decir que nuestra imagen tiene 
    # mucha iluminacion y tratamos de bajar los numero aproximandolo al valor de RGBAVG
    
    # Clase 2
    
    if ENTRYRGB >= (RGBAVG +10):
        R_VARC = (RGBAVG +10) / (statistics.mean(R_VAR)+1)
        G_VARC = (RGBAVG +10) / (statistics.mean(G_VAR)+1)
        B_VARC = (RGBAVG +10) / (statistics.mean(B_VAR)+1)

    #si nuestra imagen esta en el rango +- 10 de RGBAVG no ocupamos modificar nada y solo regresamos la 
    # imagen original
    
    # Clase 3 
    
    if ENTRYRGB >= (RGBAVG-10) and ENTRYRGB<= (RGBAVG + 10):
        print("Imagen esta en el rango acceptable no se ocupa modificar")
        return image


    RGVA = [R_VARC,G_VARC,B_VARC]
    print("Matriz a multiplicar:" + str(RGVA))
    
    image_array = np.array(image)
    # CALCULAMOS LOS NUEVOS VALORES PARA NUESTROS PIXELES
    imm = (RGVA * image_array)
    # RGB solo acepta hasta 255 si por alguna razon nos pasamos lo dejamos a 255 
    imm = np.clip(imm,0,255)

    print("RGB promedio despues de procesar:" + str(np.mean(imm)))
    # convertimos el array devuelta en imagen 
    imm = Image.fromarray(np.uint8(imm))


    return imm


## Preguntamos al usuario 1.- El Folder donde estan las imagenes a analizar y 2.- el folder donde se guardaran las imagenes procesadas

In [6]:

root = tk.Tk()
root.withdraw()

# Preguntamos por el folder donde estan nuestras imagenes que queremos procesar
path_source = filedialog.askdirectory()


# Preguntamos por el folder donde guardaremos las imagenes modificadas
path_destiny = filedialog.askdirectory()


for filename in glob.glob(path_source + '/*.png'):
   
    
    fname = os.path.basename(filename)
    final_file = (path_destiny + '/ilumination_' + fname)
    

    IMG = Image.open(filename).convert("RGB")
   

    Output = FIX_MY_IMAGE(IMG)
    Output.save(final_file)





2022-11-27 17:45:22.574 python[25014:1729593] +[CATransaction synchronize] called within transaction
2022-11-27 17:45:44.398 python[25014:1729593] +[CATransaction synchronize] called within transaction
2022-11-27 17:45:44.411 python[25014:1729593] +[CATransaction synchronize] called within transaction
2022-11-27 17:45:44.427 python[25014:1729593] +[CATransaction synchronize] called within transaction
2022-11-27 17:45:44.437 python[25014:1729593] +[CATransaction synchronize] called within transaction
2022-11-27 17:45:44.456 python[25014:1729593] +[CATransaction synchronize] called within transaction
2022-11-27 17:45:44.478 python[25014:1729593] +[CATransaction synchronize] called within transaction
2022-11-27 17:45:44.494 python[25014:1729593] +[CATransaction synchronize] called within transaction
2022-11-27 17:45:44.517 python[25014:1729593] +[CATransaction synchronize] called within transaction
2022-11-27 17:45:44.535 python[25014:1729593] +[CATransaction synchronize] called within tr

RGB promedio antes de procesar:25.888094618055554
Matriz a multiplicar:[2.084522947313247, 2.606623664766257, 2.138320558718158]
RGB promedio despues de procesar:57.02364012094241
RGB promedio antes de procesar:28.859844835069442
Matriz a multiplicar:[1.908236712486436, 2.6141598085483913, 1.7496188921303442]
RGB promedio despues de procesar:58.39895563706106
RGB promedio antes de procesar:20.551133897569443
Matriz a multiplicar:[2.5456406251128776, 2.8179915768057566, 3.1334382057431966]
RGB promedio despues de procesar:51.79177815729702
RGB promedio antes de procesar:20.79035210503472
Matriz a multiplicar:[2.2166915157327014, 2.7565577480789374, 3.776761188444571]
RGB promedio despues de procesar:54.71257923351854
RGB promedio antes de procesar:20.523088650173612
Matriz a multiplicar:[2.274343219608682, 2.825982401926274, 3.6778981727269455]
RGB promedio despues de procesar:54.38710094134361
RGB promedio antes de procesar:60.02088758680556
Matriz a multiplicar:[1.000626170452874, 1.1